### 3.1. Show 10 dòng đầu của file:
- 20251127_camera.xlsx
- 20251127_system.xlsx
- factor.xlsx
- mapping.xlsx

In [1]:
import pandas as pd

# =========================
# CONFIG
# =========================
files = {
    "CAMERA": "3/3_2/input_data/20251127_camera.xlsx",
    "SYSTEM": "3/3_2/input_data/20251127_system.xlsx",
    "MAPPING": "3/3_2/input_data/mapping.xlsx",
    "FACTOR": "3/3_2/input_data/factor.xlsx"
}

# =========================
# READ & SHOW FIRST 10 ROWS
# =========================
for name, path in files.items():
    print(f"\n===== {name} — 10 dòng đầu tiên =====")
    try:
        df = pd.read_excel(path)
        print(df.head(10))
    except Exception as e:
        print(f"Lỗi đọc file {path}: {e}")


===== CAMERA — 10 dòng đầu tiên =====
   Mã đường      Tên đường   Năm  Xe máy  Xe ô tô  Xe buýt  Xe tải nặng  \
0       101  Phố Báo Khánh  2024       0        0        0            0   
1       101  Phố Báo Khánh  2024       0        0        0            0   
2       101  Phố Báo Khánh  2024       0        0        0            0   
3       101  Phố Báo Khánh  2024       0        0        0            0   
4       101  Phố Báo Khánh  2024       0        0        0            0   
5       101  Phố Báo Khánh  2024       0        0        0            0   
6       101  Phố Báo Khánh  2024       0        0        0            0   
7       101  Phố Báo Khánh  2024     720       97      195          139   
8       101  Phố Báo Khánh  2024    1132      641      145           37   
9       101  Phố Báo Khánh  2024    1129      590      119           46   

   Xe tải nhẹ        Loại ngày    Giờ        Ngày  
0           0  Ngày trong tuần  00:00  27/11/2025  
1           0  Ngày trong tuần 

#### 3.2. Nội suy lưu và sinh lưu lượng phương tiện của những khung giờ không có lưu lượng phương tiện của mỗi đường

In [10]:
import os
import pandas as pd
import numpy as np

# ==========================================================
# 1) CONFIG
# ==========================================================
input_dir = "3/3_2/input_data_1"
output_dir = "3/3_2/output_data_1"
os.makedirs(output_dir, exist_ok=True)

camera_file  = os.path.join(input_dir, "20251127_camera.xlsx")
mapping_file = os.path.join(input_dir, "mapping.xlsx")
factor_file  = os.path.join(input_dir, "factor.xlsx")
output_file = os.path.join(output_dir, "20251127_camera_interpolated.xlsx")

# ==========================================================
# 2) LOAD DATA
# ==========================================================
camera_df = pd.read_excel(camera_file)
factor_df = pd.read_excel(factor_file)
mapping_df = pd.read_excel(mapping_file)

# ==========================================================
# 3) CALCULATE AVERAGE TRAFFIC VOLUME PER HOUR
# ==========================================================
# Group by Road Code, Road Name, Year, and calculate the average for each hour
average_traffic_df = camera_df.groupby(['Mã đường', 'Tên đường', 'Năm', 'Loại ngày', 'Ngày', 'Giờ']).agg({
    'Xe máy': 'mean',
    'Xe ô tô': 'mean',
    'Xe buýt': 'mean',
    'Xe tải nặng': 'mean',
    'Xe tải nhẹ': 'mean'
}).reset_index()

# Calculate the overall averages per road and day
average_traffic_overall_df = average_traffic_df.groupby(['Mã đường', 'Tên đường', 'Năm', 'Loại ngày']).agg({
    'Xe máy': 'mean',
    'Xe ô tô': 'mean',
    'Xe buýt': 'mean',
    'Xe tải nặng': 'mean',
    'Xe tải nhẹ': 'mean'
}).reset_index()

# ==========================================================
# 4) FACTOR CALCULATIONS FOR INTERPOLATION
# ==========================================================
# Merge factor values with average traffic volumes to create interpolation ratios
factor_df['Giá trị'] = factor_df['Giá trị'].astype(float)
factor_pivot_df = factor_df.pivot(index='Giờ', columns='Loại phương tiện', values='Giá trị').reset_index()

# For each missing hour in the camera data, we will interpolate using these factors
def interpolate_missing_hours(row, factor_pivot_df, average_traffic_overall_df):
    # Get the average traffic values for the road and day
    avg_traffic = average_traffic_overall_df[
        (average_traffic_overall_df['Mã đường'] == row['Mã đường']) &
        (average_traffic_overall_df['Tên đường'] == row['Tên đường']) &
        (average_traffic_overall_df['Năm'] == row['Năm']) &
        (average_traffic_overall_df['Loại ngày'] == row['Loại ngày'])
    ]

    # If the row's traffic volume is 0, interpolate using the average and factor
    if row['Xe máy'] == 0:
        factor = factor_pivot_df[factor_pivot_df['Giờ'] == row['Giờ']]['Xe máy'].values[0]
        row['Xe máy'] = avg_traffic['Xe máy'].values[0] * factor

    if row['Xe ô tô'] == 0:
        factor = factor_pivot_df[factor_pivot_df['Giờ'] == row['Giờ']]['Xe ô tô'].values[0]
        row['Xe ô tô'] = avg_traffic['Xe ô tô'].values[0] * factor

    if row['Xe buýt'] == 0:
        factor = factor_pivot_df[factor_pivot_df['Giờ'] == row['Giờ']]['Xe buýt'].values[0]
        row['Xe buýt'] = avg_traffic['Xe buýt'].values[0] * factor

    if row['Xe tải nặng'] == 0:
        factor = factor_pivot_df[factor_pivot_df['Giờ'] == row['Giờ']]['Xe tải nặng'].values[0]
        row['Xe tải nặng'] = avg_traffic['Xe tải nặng'].values[0] * factor

    if row['Xe tải nhẹ'] == 0:
        factor = factor_pivot_df[factor_pivot_df['Giờ'] == row['Giờ']]['Xe tải nhẹ'].values[0]
        row['Xe tải nhẹ'] = avg_traffic['Xe tải nhẹ'].values[0] * factor

    return row

# Apply the interpolation for each row in the camera dataframe
camera_df = camera_df.apply(interpolate_missing_hours, axis=1, factor_pivot_df=factor_pivot_df, average_traffic_overall_df=average_traffic_overall_df)

# ==========================================================
# 5) SAVE OUTPUT
# ==========================================================
camera_df.to_excel(output_file, index=False)

print(f"File with interpolated traffic data saved as: {output_file}")


File with interpolated traffic data saved as: 3/3_2/output_data_2/20251127_camera_interpolated.xlsx


### 3.3. Ánh xạ lưu lượng phương tiện từ đường camera giao thông thực tế => đoạn đường ở trong CSDL.
- Case 2: Đường Camera không đầy đủ dữ liệu của khung 24h (Note: chỉ có một số khung giờ có lưu lượng phương tiện).
- Nhưng làm tương tự case 1 vì file 20251127_camera_interpolated.xlsx đã đầy đủ nhờ bước thứ nhất

In [17]:
import os
import pandas as pd

# ==========================================================
# 1) CONFIG
# ==========================================================
input_dir = "3/3_2/2_input_data"
output_dir = "3/3_2/2_output_data"
os.makedirs(output_dir, exist_ok=True)

camera_file  = os.path.join(input_dir, "20251127_camera_interpolated.xlsx")
system_file  = os.path.join(input_dir, "20251127_system.xlsx")
mapping_file = os.path.join(input_dir, "mapping.xlsx")

output_file = os.path.join(output_dir, "20251127_system_mapped.xlsx")


# ==========================================================
# 2) LOAD DATA
# ==========================================================
print("=== Đang đọc file camera... ===")
df_camera = pd.read_excel(camera_file)

print("=== Đang đọc file system... ===")
df_system = pd.read_excel(system_file)

print("=== Đang đọc file mapping... ===")
df_mapping = pd.read_excel(mapping_file)

print(">>> Đọc file hoàn tất\n")


# ==========================================================
# 3) TIỀN XỬ LÝ
# ==========================================================
# Chuẩn hóa kiểu dữ liệu
df_camera["Mã đường"] = df_camera["Mã đường"].astype(str)
df_system["Mã đường"] = df_system["Mã đường"].astype(str)
df_mapping["Mã đường"] = df_mapping["Mã đường"].astype(str)
df_mapping["Mã đoạn đường"] = df_mapping["Mã đoạn đường"].astype(str)

# Tạo dict ánh xạ: road_code → list(segment_code)
map_dict = (
    df_mapping.groupby("Mã đường")["Mã đoạn đường"]
    .apply(list)
    .to_dict()
)

print("=== Sinh bảng ánh xạ hoàn tất ===\n")


# ==========================================================
# 4) CHUẨN HÓA CAMERA THEO GIỜ
# ==========================================================
# Đảm bảo group đúng theo từng Mã đường + Giờ
camera_grouped = df_camera.groupby(["Mã đường", "Giờ"], as_index=False).agg({
    "Xe máy": "sum",
    "Xe ô tô": "sum",
    "Xe buýt": "sum",
    "Xe tải nặng": "sum",
    "Xe tải nhẹ": "sum"
})

print("=== Gom nhóm camera theo Mã đường + Giờ hoàn tất ===\n")


# ==========================================================
# 5) MAP DỮ LIỆU CAMERA → SYSTEM
# ==========================================================
print("=== Bắt đầu ánh xạ dữ liệu camera vào system ===")

# Explicitly cast relevant columns to float to avoid dtype conflict
df_system["Xe máy"] = df_system["Xe máy"].astype(float)
df_system["Xe ô tô"] = df_system["Xe ô tô"].astype(float)
df_system["Xe buýt"] = df_system["Xe buýt"].astype(float)
df_system["Xe tải nặng"] = df_system["Xe tải nặng"].astype(float)
df_system["Xe tải nhẹ"] = df_system["Xe tải nhẹ"].astype(float)

records_updated = 0

for idx, row in df_system.iterrows():
    segment_code = row["Mã đường"]          # ví dụ: 1000561
    hour = row["Giờ"]                       # ví dụ: 07:00

    # Tìm mã đường gốc camera
    found = None
    for road, segment_list in map_dict.items():
        if segment_code in segment_list:
            found = road
            break

    if not found:
        # Không ánh xạ → bỏ qua
        continue

    # Lấy dữ liệu camera tương ứng Mã đường + Giờ
    matched = camera_grouped[
        (camera_grouped["Mã đường"] == found) &
        (camera_grouped["Giờ"] == hour)
    ]

    if matched.empty:
        continue

    camera_row = matched.iloc[0]

    # Ghi dữ liệu vào system (values are already floats)
    df_system.at[idx, "Xe máy"] = camera_row["Xe máy"]
    df_system.at[idx, "Xe ô tô"] = camera_row["Xe ô tô"]
    df_system.at[idx, "Xe buýt"] = camera_row["Xe buýt"]
    df_system.at[idx, "Xe tải nặng"] = camera_row["Xe tải nặng"]
    df_system.at[idx, "Xe tải nhẹ"] = camera_row["Xe tải nhẹ"]

    records_updated += 1

print(f"=== Hoàn tất ánh xạ. Tổng số record được cập nhật: {records_updated} ===\n")


# ==========================================================
# 6) EXPORT FILE
# ==========================================================
df_system.to_excel(output_file, index=False)
print(f"=== ĐÃ XUẤT FILE HOÀN TẤT: {output_file} ===")

=== Đang đọc file camera... ===
=== Đang đọc file system... ===
=== Đang đọc file mapping... ===
>>> Đọc file hoàn tất

=== Sinh bảng ánh xạ hoàn tất ===

=== Gom nhóm camera theo Mã đường + Giờ hoàn tất ===

=== Bắt đầu ánh xạ dữ liệu camera vào system ===
=== Hoàn tất ánh xạ. Tổng số record được cập nhật: 7656 ===

=== ĐÃ XUẤT FILE HOÀN TẤT: 3/3_2/2_output_data/20251127_system_mapped.xlsx ===
